In [1]:
import sqlite3
import pandas as pd
import datetime
# Importing awake data gathering script
from gather_awake_data import main
# Importing asleep data gathering script
import gather_asleep_data

In [2]:
# Initialize SQLite connection and cursor
conn = sqlite3.connect('database.db')
c = conn.cursor()

# Create a table if it doesn't exist
c.execute('''CREATE TABLE IF NOT EXISTS data (
                Date TEXT PRIMARY KEY,
                SleepQuality INTEGER,
                MealQuality INTEGER,
                EnergyLevel INTEGER,
                WeatherQual INTEGER,
                Socialization INTEGER,
                JobHours INTEGER,
                WorkProd INTEGER,
                ChoreProd INTEGER,
                WorkOut INTEGER,
                Weight INTEGER,
                GoOutside INTEGER,
                Shower INTEGER,
                Caffeine INTEGER,
                PartnerHours INTEGER,
                GamingHours INTEGER,
                TVHours INTEGER,
                AppHours INTEGER,
                StressLevel INTEGER,
                DayRating INTEGER,
                EODEmotion TEXT,
                DayOfWeek TEXT,
                AwakeCount INTEGER,
                AwakeDuration INTEGER,
                AwakeningsCount INTEGER,
                Duration INTEGER,
                Efficiency INTEGER,
                EndTime TEXT,
                IsMainSleep INTEGER,
                LogId TEXT,
                MinutesAfterWakeup INTEGER,
                MinutesAsleep INTEGER,
                MinutesAwake INTEGER,
                MinutesToFallAsleep INTEGER,
                RestlessCount INTEGER,
                RestlessDuration INTEGER,
                StartTime TEXT,
                TimeInBed INTEGER
            )''')

In [3]:
# Importing awake data and preprocessing
raw_awake_df = main()

# Creating a copy to avoid overusing the API
awake_df = raw_awake_df.copy()

# Splitting the 'Date' column into 'DayOfWeek' and 'Date' columns
awake_df[['DayOfWeek', 'Date']] = awake_df['Date'].str.split(', ', expand=True)

# Convert 'Date' column to datetime format with the specified format
awake_df['Date'] = pd.to_datetime(awake_df['Date'], format='%m/%d/%y')

# Read the most recent date from the SQLite database
c.execute("SELECT MAX(Date) FROM data")
latest_date = c.fetchone()[0]
if latest_date is None:
    latest_date = '2024-01-01'

# Selecting new awake data
new_awake_df = awake_df[awake_df['Date'] > latest_date]

In [4]:
# Fetching and formatting sleep data
start_date = (pd.to_datetime(latest_date) + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
end_date = datetime.datetime.now().strftime('%Y-%m-%d')

# Define Fitbit credentials
client_id = ""
client_secret = ""
redirect_uri = '/'

# Instantiate OAuth server
oauth_server = gather_asleep_data.OAuth2Server(client_id, client_secret, redirect_uri)
oauth_server.browser_authorize()

# Access token and refresh token are now available through oauth_server.fitbit.client.session.token
access_token = oauth_server.fitbit.client.session.token['access_token']
refresh_token = oauth_server.fitbit.client.session.token['refresh_token']

# Create an authorized Fitbit API client
fitbit_client = gather_asleep_data.FitbitAPI(client_id, client_secret, access_token, refresh_token, redirect_uri).get_authorized_client()

# Fetch and format sleep data
raw_sleep_df = gather_asleep_data.fetch_and_format_sleep_data(start_date, end_date, fitbit_client)
raw_sleep_df.head()

[20/Apr/2024:20:26:42] ENGINE Listening for SIGTERM.
[20/Apr/2024:20:26:42] ENGINE Listening for SIGHUP.
[20/Apr/2024:20:26:42] ENGINE Listening for SIGUSR1.
[20/Apr/2024:20:26:42] ENGINE Bus STARTING
CherryPy Checker:
The Application mounted at '' has an empty config.

[20/Apr/2024:20:26:42] ENGINE Started monitor thread 'Autoreloader'.
[20/Apr/2024:20:26:42] ENGINE Serving on http://127.0.0.1:8080
[20/Apr/2024:20:26:42] ENGINE Bus STARTED


127.0.0.1 - - [20/Apr/2024:20:26:43] "GET /?code=26361440bee14451065e5a696eeb6bce17ca2603&state=dQckbhtTLOtX7KDsEZzA1dDsJHIE8S HTTP/1.1" 200 114 "" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"


[20/Apr/2024:20:26:44] ENGINE Bus STOPPING
[20/Apr/2024:20:26:44] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8080)) shut down
[20/Apr/2024:20:26:44] ENGINE Stopped thread 'Autoreloader'.
[20/Apr/2024:20:26:44] ENGINE Bus STOPPED
[20/Apr/2024:20:26:44] ENGINE Bus EXITING
[20/Apr/2024:20:26:44] ENGINE Bus EXITED
[20/Apr/2024:20:26:44] ENGINE Waiting for child threads to terminate...


""


In [ ]:
# Creating a copy to avoid overusing the API
new_asleep_df = raw_sleep_df.copy()
new_asleep_df.rename(columns={'date': 'Date'}, inplace=True)
new_asleep_df['Date'] = pd.to_datetime(new_asleep_df['Date'])

In [ ]:
# Merge awake and asleep data
merged_df = pd.merge(new_awake_df, new_sleep_df, on='Date', how='outer')

# Merge with existing data from the database, keeping only new rows
c.execute("SELECT * FROM data")
database_df = pd.DataFrame(c.fetchall(), columns=['Date', 'DayOfWeek'])  # Include columns you need here
new_data = pd.concat([database_df, merged_df]).drop_duplicates(subset=['Date'], keep='last')

# Drop empty rows
merged_df = merged_df.dropna(how='all')

# Save the updated DataFrame to SQLite database
new_data.to_sql('data', conn, if_exists='replace', index=False)

# Commit changes and close the connection
conn.commit()
conn.close()